<a href="https://colab.research.google.com/github/edwardhan925192/ML-Deep-Learning/blob/main/Vision/SimpleCNN_covid19model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D,GlobalAveragePooling2D
from keras.layers import Activation, Dropout, BatchNormalization, Flatten, Dense, AvgPool2D,MaxPool2D
from keras.models import Sequential, Model
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.optimizers import Adam, SGD, RMSprop

import tensorflow as tf

import os
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import pathlib 			 	        #Path Object로 만들어줌 
import PIL.Image as image 	 	#jpg를 screen에서 보여준다. 
import cv2

In [39]:
from sklearn.model_selection import train_test_split

In [31]:
Image_shape = (244,244)

In [3]:
covid_data = pathlib.Path('/content/drive/MyDrive/data/covid data')

In [9]:
covid_dict = {}

In [6]:
covid_list = []
for filename in covid_data.iterdir():
  covid_list.append(filename.name)

In [10]:
for idx,name in enumerate(covid_list):
  covid_dict[name] = idx

In [34]:
labels = []
X_value = []
for filename in covid_data.iterdir():
  temp = filename.name
  for image in list(filename.glob('*.[jp][pn][g]')):    
    img = cv2.imread(str(image))
    X_value.append(cv2.resize(img,Image_shape))

    labels.append(covid_dict[temp])

In [43]:
X_scaled = np.array(X_value)/255

In [44]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, labels, random_state=0)

#X_value = resized img data 
#labels = labels 

In [41]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(244,244,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Conv2D(64,(3,3)))
model.add(Activation("relu"))
model.add(Conv2D(250,(3,3)))
model.add(Activation("relu"))
  
model.add(Conv2D(128,(3,3)))
model.add(Activation("relu"))
model.add(AvgPool2D(2,2))
model.add(Conv2D(64,(3,3)))
model.add(Activation("relu"))
model.add(AvgPool2D(2,2))

model.add(Conv2D(256,(2,2)))
model.add(Activation("relu"))
model.add(MaxPool2D(2,2))
    
model.add(Flatten())
model.add(Dense(32))
model.add(Dropout(0.25))
model.add(Dense(1))
model.add(Activation("sigmoid"))

In [45]:
train_datagen = ImageDataGenerator(rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.3)

train_generator = train_datagen.flow_from_directory(
    covid_data,
    target_size=Image_shape,
    batch_size = 6,
    class_mode='binary',
    subset='training')

validation_generator = train_datagen.flow_from_directory(
    covid_data, 
    target_size=Image_shape,
    batch_size = 6,
    class_mode='binary',
    shuffle= False,
    subset='validation')

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

Found 560 images belonging to 2 classes.
Found 240 images belonging to 2 classes.


In [46]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.samples // 6,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // 6,
    epochs = 48)

<ipython-input-46-d3c2a27db6a0>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


Epoch 1/48
93/93 [==============================] - 220s 2s/step - loss: 0.7161 - accuracy: 0.5000 - val_loss: 0.7071 - val_accuracy: 0.5000
Epoch 2/48
93/93 [==============================] - ETA: 0s - loss: 0.7277 - accuracy: 0.5451

KeyboardInterrupt: ignored

In [ ]:
image.shape

(1237, 1156, 3)